In [1]:
import cobra

In [2]:
cobra_model = cobra.io.read_sbml_model('W_La_Ca_iIA409_20Z.xml')
cobra_model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.001502,0,0.00%
ch4_e,EX_ch4_e,11.7,1,100.00%
cu2_e,EX_cu2_e,0.001502,0,0.00%
fe2_e,EX_fe2_e,0.000919,0,0.00%
mg2_e,EX_mg2_e,0.01051,0,0.00%
no3_e,EX_no3_e,1.019,0,0.00%
o2_e,EX_o2_e,13.77,0,0.00%
pi_e,EX_pi_e,0.1608,0,0.00%
so4_e,EX_so4_e,0.01772,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux


## Psilocybin Pathway Addition

In [3]:
# PsiD
r_psiD = cobra.Reaction("TRPDC")
r_psiD.name = "L-tryptophan decarboxylase"
r_psiD.subsystem = "Tryptophan Metabolism"
r_psiD.lower_bound = 0
r_psiD.upper_bound = 1000

trpmine = cobra.Metabolite(
    'trpmine_c',
    formula='C10H12N2',
    name='Tryptamine',
    compartment='c')

ltrp = cobra_model.metabolites.get_by_id("ltrp_c")
co2_c = cobra_model.metabolites.get_by_id("co2_c")

r_psiD.add_metabolites({
    ltrp: -1.0,
    trpmine: 1.0,
    co2_c: 1.0
})

In [4]:
# PsiH
r_psiH = cobra.Reaction("TRPMINE4MO")
r_psiH.name = "Tryptamine 4-monooxygenase"
r_psiH.subsystem = "Tryptophan Metabolism"
r_psiH.lower_bound = 0
r_psiH.upper_bound = 1000

htrpmine = cobra.Metabolite(
    'htrpmine_c',
    formula='C10H12N2O',
    name='4-Hydroxytryptamine',
    compartment='c')

o2_c = cobra_model.metabolites.get_by_id("o2_c")
h2o_c = cobra_model.metabolites.get_by_id("h2o_c")
elacrd_c = cobra_model.metabolites.get_by_id("elacrd_c")
elaco_c = cobra_model.metabolites.get_by_id("elaco_c")


r_psiH.add_metabolites({
    trpmine: -1.0,
    elacrd_c: -1.0,
    o2_c: -1.0,
    htrpmine: 1.0,
    elaco_c: 1.0,
    h2o_c: 1.0
})


In [5]:
# PsiK
r_psiK1 = cobra.Reaction("4HTRPMINEK")
r_psiK1.name = "4-hydroxytryptamine kinase"
r_psiK1.subsystem = "Tryptophan Metabolism"
r_psiK1.lower_bound = 0
r_psiK1.upper_bound = 1000

nbaecys = cobra.Metabolite(
    'nbaecys_c',
    formula='C10H13N2O4P',
    name='Norbaeocystin',
    compartment='c')

atp_c = cobra_model.metabolites.get_by_id("atp_c")
adp_c = cobra_model.metabolites.get_by_id("adp_c")

r_psiK1.add_metabolites({
    atp_c: -1.0,
    htrpmine: -1.0,
    adp_c: 1.0,
    nbaecys: 1.0
})



In [6]:
# PsiM
r_psiM1 = cobra.Reaction("PSIBINS1")
r_psiM1.name = "SAM:Norbaeocystin N-methyltransferase"
r_psiM1.subsystem = "Tryptophan Metabolism"
r_psiM1.lower_bound = 0
r_psiM1.upper_bound = 1000

baecys = cobra.Metabolite(
    'baecys_c',
    formula='C11H15N2O4P',
    name='Baeocystin',
    compartment='c')

amet_c = cobra_model.metabolites.get_by_id("amet_c")
ahcys_c = cobra_model.metabolites.get_by_id("ahcys_c")

r_psiM1.add_metabolites({
    amet_c: -1.0,
    nbaecys: -1.0,
    ahcys_c: 1.0,
    baecys: 1.0
})

r_psiM2 = cobra.Reaction("PSIBINS2")
r_psiM2.name = "Psilocybin Synthase"
r_psiM2.subsystem = "Tryptophan Metabolism"
r_psiM2.lower_bound = 0
r_psiM2.upper_bound = 1000

psibin = cobra.Metabolite(
    'psibin_c',
    formula='C12H17N2O4P',
    name='Psilocybin',
    compartment='c')

r_psiM2.add_metabolites({
    amet_c: -1.0,
    baecys: -1.0,
    ahcys_c: 1.0,
    psibin: 1.0
})


In [7]:
cobra_model.add_reactions([r_psiD,r_psiH,r_psiK1,r_psiM1,r_psiM2])
cobra_model.add_boundary(cobra_model.metabolites.get_by_id("psibin_c"),type="demand")

Reaction identifier,DM_psibin_c
Name,Psilocybin demand
Memory address,0x7fc4ad570f50
Stoichiometry,psibin_c --> Psilocybin -->
GPR,
Lower bound,0
Upper bound,1000.0


In [8]:
cobra_model.objective = 'PSIBINS2'
cobra_model.metabolites.get_by_id('ltrp_c').summary()

Percent,Flux,Reaction,Definition
100.00%,0.5847,TRPS1,3ig3p_c + lser_c --> g3p_c + h2o_c + ltrp_c
Percent,Flux,Reaction,Definition
100.00%,-0.5847,TRPDC,ltrp_c --> co2_c + trpmine_c


In [9]:
for r in cobra_model.reactions:
    if r.subsystem == 'Tryptophan Metabolism':
        print(r)

TRPDC: ltrp_c --> co2_c + trpmine_c
TRPMINE4MO: elacrd_c + o2_c + trpmine_c --> elaco_c + h2o_c + htrpmine_c
4HTRPMINEK: atp_c + htrpmine_c --> adp_c + nbaecys_c
PSIBINS1: amet_c + nbaecys_c --> ahcys_c + baecys_c
PSIBINS2: amet_c + baecys_c --> ahcys_c + psibin_c


In [10]:
cobra_model.reactions.PSIBINS2.flux

0.5846639862147948